<a href="https://colab.research.google.com/github/stwater20/AIS3-2024-Material/blob/main/AIS3_Lab1_threat_entity_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install datasets seqeval evaluate huggingface transformers accelerate>=0.21.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [5]:
from datasets import Dataset, DatasetDict, ClassLabel, Sequence, Features
import pandas as pd

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [19]:
def load_data(file_path):
  tokens = []
  ner_tags = []
  temp_tokens = []
  temp_ner_tags = []
  with open(file_path, 'r', encoding='utf-8') as file:
      for line in file:
          if len(line.split())>1:
              line = line.strip()
              token, tag = line.rsplit(' ', 1)
              temp_tokens.append(token)
              temp_ner_tags.append(tag)
          else:
              tokens.append(temp_tokens)
              ner_tags.append(temp_ner_tags)
              temp_tokens = []
              temp_ner_tags = []
  ds = Dataset.from_dict({'tokens': tokens, 'ner_tags': ner_tags})
  return ds

ds_train = load_data('DNRTI/train.txt')
ds_valid = load_data('DNRTI/valid.txt')
ds_test = load_data('DNRTI/test.txt')

In [20]:
unique_tags = set()
for dataset in [ds_train, ds_valid, ds_test]:
    unique_tags.update([tag for sublist in dataset['ner_tags'] for tag in sublist])
unique_tags = sorted(list(unique_tags))
# unique_tags

In [21]:
label2id = {label: idx for idx, label in enumerate(unique_tags)}
id2label = {idx: label for label, idx in label2id.items()}

In [22]:
def update_ner_tags(example):
    return {'ner_tags': [label2id[tag] for tag in example['ner_tags']]}

ds_train = ds_train.map(update_ner_tags)
ds_valid = ds_valid.map(update_ner_tags)
ds_test = ds_test.map(update_ner_tags)

Map:   0%|          | 0/5439 [00:00<?, ? examples/s]

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

Map:   0%|          | 0/679 [00:00<?, ? examples/s]

In [23]:
raw_datasets = DatasetDict({
    'train': ds_train,
    'valid': ds_valid,
    'test': ds_test
})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 5439
    })
    valid: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 668
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 679
    })
})

In [24]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = id2label[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

The admin@338 has largely targeted organizations involved in financial , economic and trade  policy , typically using publicly available RATs   such as Poison Ivy    , as well some non-public backdoors . 
O   B-HackOrg O   O       O        O             O        O  B-Idus    O B-Idus   O   B-Idus I-Idus O O         O     B-Tool   I-Tool    I-Tool O    O  B-Tool I-Tool O O  O    O    B-Tool     I-Tool    O 


In [25]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [26]:
tokenizer.is_fast

True

In [27]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()[:5]

['[CLS]', 'The', 'ad', '##min', '@']

In [28]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [29]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[26, 3, 26, 26, 26, 26, 26, 26, 4, 26, 4, 26, 4, 17, 26, 26, 26, 11, 24, 24, 26, 26, 11, 24, 26, 26, 26, 26, 11, 24, 26]
[-100, 26, 3, 4, 4, 4, 4, 26, 26, 26, 26, 26, 26, 4, 26, 4, 26, 4, 17, 26, 26, 26, 11, 24, 24, 24, 24, 26, 26, 11, 12, 24, 26, 26, 26, 26, 11, 12, 12, 24, 24, 24, 26, -100]


In [30]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [31]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/5439 [00:00<?, ? examples/s]

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

Map:   0%|          | 0/679 [00:00<?, ? examples/s]

In [32]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [33]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,   26,    3,    4,    4,    4,    4,   26,   26,   26,   26,   26,
           26,    4,   26,    4,   26,    4,   17,   26,   26,   26,   11,   24,
           24,   24,   24,   26,   26,   11,   12,   24,   26,   26,   26,   26,
           11,   12,   12,   24,   24,   24,   26, -100],
        [-100,   26,    3,    4,    4,    4,    4,   26,   26,    0,   13,    6,
           19,   26,   26,   26,   26,   26,    4,   26,    4,   26,   26,    0,
           13,   26,    0,   26, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100]])

In [34]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 26, 3, 4, 4, 4, 4, 26, 26, 26, 26, 26, 26, 4, 26, 4, 26, 4, 17, 26, 26, 26, 11, 24, 24, 24, 24, 26, 26, 11, 12, 24, 26, 26, 26, 26, 11, 12, 12, 24, 24, 24, 26, -100]
[-100, 26, 3, 4, 4, 4, 4, 26, 26, 0, 13, 6, 19, 26, 26, 26, 26, 26, 4, 26, 4, 26, 26, 0, 13, 26, 0, 26, -100]


In [35]:
import evaluate

metric = evaluate.load("seqeval")

In [36]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [id2label[i] for i in labels]
labels

['O',
 'B-HackOrg',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Idus',
 'O',
 'B-Idus',
 'O',
 'B-Idus',
 'I-Idus',
 'O',
 'O',
 'O',
 'B-Tool',
 'I-Tool',
 'I-Tool',
 'O',
 'O',
 'B-Tool',
 'I-Tool',
 'O',
 'O',
 'O',
 'O',
 'B-Tool',
 'I-Tool',
 'O']

In [37]:
predictions = labels.copy()
predictions[1] = "O"
metric.compute(predictions=[predictions], references=[labels])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'HackOrg': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'Idus': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'Tool': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'overall_precision': 1.0,
 'overall_recall': 0.8571428571428571,
 'overall_f1': 0.923076923076923,
 'overall_accuracy': 0.967741935483871}

In [38]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [39]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
model.config.num_labels

27

In [46]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [47]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.216100,0.425843,0.744448,0.809937,0.775813,0.885162
2,0.155400,0.410723,0.780098,0.813680,0.796535,0.898921
3,0.112200,0.432195,0.779176,0.853156,0.814490,0.905278
4,0.090400,0.394568,0.820247,0.860303,0.839797,0.916778
5,0.075300,0.423848,0.828191,0.867790,0.847528,0.921546
6,0.046500,0.450445,0.827496,0.864387,0.845539,0.921337
7,0.037400,0.460288,0.839189,0.873745,0.856119,0.924515
8,0.035100,0.485335,0.833874,0.874596,0.853750,0.923888
9,0.022100,0.506148,0.835951,0.878339,0.856621,0.924473
10,0.019400,0.498738,0.840176,0.877489,0.858427,0.926522


TrainOutput(global_step=6800, training_loss=0.07607460772289949, metrics={'train_runtime': 1064.3367, 'train_samples_per_second': 51.102, 'train_steps_per_second': 6.389, 'total_flos': 1629316148192190.0, 'train_loss': 0.07607460772289949, 'epoch': 10.0})

In [48]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/ShengShan/bert-finetuned-ner/commit/4be6557a05cf060f8d15efd1b17e14c3b6ee4c4b', commit_message='Training complete', commit_description='', oid='4be6557a05cf060f8d15efd1b17e14c3b6ee4c4b', pr_url=None, pr_revision=None, pr_num=None)

In [49]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.31583818793296814,
 'eval_precision': 0.8729711825107651,
 'eval_recall': 0.9036516372364135,
 'eval_f1': 0.8880464998736416,
 'eval_accuracy': 0.9478101567825494,
 'eval_runtime': 3.3345,
 'eval_samples_per_second': 203.629,
 'eval_steps_per_second': 25.491,
 'epoch': 10.0}

In [50]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "ShengShan/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier(["Eset‍ has published a report on the state-sponsored Russian turla apt group."])

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

[[{'entity_group': 'SecTeam',
   'score': 0.99967444,
   'word': 'E',
   'start': 0,
   'end': 1},
  {'entity_group': 'Time',
   'score': 0.9996977,
   'word': '##set',
   'start': 1,
   'end': 4},
  {'entity_group': 'Area',
   'score': 0.8720983,
   'word': 'Russian',
   'start': 52,
   'end': 59},
  {'entity_group': 'HackOrg',
   'score': 0.9973858,
   'word': 't',
   'start': 60,
   'end': 61},
  {'entity_group': 'Idus',
   'score': 0.9656144,
   'word': '##ur',
   'start': 61,
   'end': 63},
  {'entity_group': 'Idus',
   'score': 0.9639422,
   'word': '##la',
   'start': 63,
   'end': 65}]]